In [13]:
import pygame
import random

# เริ่มต้น Pygame
pygame.init()

# ขนาดหน้าจอ
screen_width = 700
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

# สี
WHITE = (255, 255, 255)
LINE_COLOR = (0, 0, 0)
RED = (242, 85, 96)
YELLOW = (255, 255, 0)
BG_COLOR = (173, 216, 230)  # สีฟ้าอ่อน

# ขนาดตาราง
ROW_COUNT = 6
COLUMN_COUNT = 7
RADIUS = 50
DIAMETER = RADIUS * 2
SQUARESIZE = 100
MARGIN = 10

# ฟอนต์
font = pygame.font.SysFont(None, 50)

# ตัวแปรเกม
board = [["" for _ in range(COLUMN_COUNT)] for _ in range(ROW_COUNT)]  # ตาราง 6x7
player_turn = True  # True หมายถึงผู้เล่น, False หมายถึงบอท
game_over = False

# ฟังก์ชันเพื่อวาดตาราง
def draw_grid():
    screen.fill(BG_COLOR)
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, LINE_COLOR, (c * SQUARESIZE + MARGIN, r * SQUARESIZE + MARGIN, DIAMETER, DIAMETER), 2)

# ฟังก์ชันเพื่อวาดเหรียญ
def draw_coins():
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == "X":
                pygame.draw.circle(screen, RED, (c * SQUARESIZE + MARGIN + RADIUS, r * SQUARESIZE + MARGIN + RADIUS), RADIUS)
            elif board[r][c] == "O":
                pygame.draw.circle(screen, YELLOW, (c * SQUARESIZE + MARGIN + RADIUS, r * SQUARESIZE + MARGIN + RADIUS), RADIUS)

# ฟังก์ชันตรวจสอบว่าแถวไหนว่างหรือไม่
def drop_piece(col, piece):
    for row in range(ROW_COUNT-1, -1, -1):
        if board[row][col] == "":
            board[row][col] = piece
            return row, col
    return None

# ฟังก์ชันตรวจสอบว่าใครชนะหรือไม่
def check_win(piece):
    # แนวนอน
    for r in range(ROW_COUNT):
        for c in range(COLUMN_COUNT-3):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True
    # แนวตั้ง
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT-3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True
    # แนวทแยง (จากซ้ายไปขวา)
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True
    # แนวทแยง (จากขวาไปซ้าย)
    for r in range(3, ROW_COUNT):
        for c in range(COLUMN_COUNT-3):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True
    return False

# ฟังก์ชันการทำงานของบอทด้วย Minimax Algorithm + Alpha-Beta Pruning
def bot_move():
    best_score = -float('inf')
    best_col = None
    for col in range(COLUMN_COUNT):
        if board[0][col] == "":
            row, col_played = drop_piece(col, "O")
            score = minimax(board, 4, False, -float('inf'), float('inf'))
            undo_move(row, col_played)
            if score > best_score:
                best_score = score
                best_col = col
    return best_col

# ฟังก์ชัน Minimax Algorithm with Alpha-Beta Pruning
def minimax(board, depth, is_maximizing, alpha, beta):
    if check_win("O"):
        return 1
    if check_win("X"):
        return -1
    if depth == 0 or is_board_full(board):
        return 0

    if is_maximizing:
        best_score = -float('inf')
        for col in range(COLUMN_COUNT):
            if board[0][col] == "":
                row, col_played = drop_piece(col, "O")
                score = minimax(board, depth - 1, False, alpha, beta)
                undo_move(row, col_played)
                best_score = max(score, best_score)
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break  # Alpha-Beta Pruning
        return best_score
    else:
        best_score = float('inf')
        for col in range(COLUMN_COUNT):
            if board[0][col] == "":
                row, col_played = drop_piece(col, "X")
                score = minimax(board, depth - 1, True, alpha, beta)
                undo_move(row, col_played)
                best_score = min(score, best_score)
                beta = min(beta, best_score)
                if beta <= alpha:
                    break  # Alpha-Beta Pruning
        return best_score

# ฟังก์ชันตรวจสอบว่าตารางเต็มหรือไม่
def is_board_full(board):
    for r in range(ROW_COUNT):
        for c in range(COLUMN_COUNT):
            if board[r][c] == "":
                return False
    return True

# ฟังก์ชันย้อนกลับการเล่น
def undo_move(row, col):
    board[row][col] = ""

# ฟังก์ชันการวาดข้อความ
def draw_text(text, font, color, x, y):
    textobj = font.render(text, 1, color)
    screen.blit(textobj, (x, y))

# เกมหลัก
def game_loop():
    global player_turn, game_over

    while True:
        draw_grid()
        draw_coins()

        # ตรวจสอบผลลัพธ์
        if check_win("X"):
            game_over = True
            draw_text("Player X Wins!", font, LINE_COLOR, 250, 250)
            pygame.display.update()
            pygame.time.delay(2000)
            break
        elif check_win("O"):
            game_over = True
            draw_text("Bot O Wins!", font, LINE_COLOR, 250, 250)
            pygame.display.update()
            pygame.time.delay(2000)
            break

        # ตรวจสอบเหตุการณ์
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

            if event.type == pygame.MOUSEBUTTONDOWN and not game_over:
                if player_turn:
                    x_pos = event.pos[0]
                    col = x_pos // SQUARESIZE  # คำนวณคอลัมน์จากตำแหน่ง X ที่คลิก
                    if board[0][col] == "":  # ตรวจสอบว่าคอลัมน์นั้นว่างหรือไม่
                        drop_piece(col, "X")
                        player_turn = False  # เปลี่ยนเป็นตาของบอท
                        pygame.display.update()

        # ให้บอทเล่นหลังจากผู้เล่น
        if not player_turn and not game_over:
            col = bot_move()
            drop_piece(col, "O")
            player_turn = True  # เปลี่ยนเป็นตาของผู้เล่น
            pygame.display.update()

        pygame.display.update()
        pygame.time.Clock().tick(30)

# เริ่มเกม
game_loop()

pygame.quit()
